In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import psycopg2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from antenna_optimization.utils import tools_new_method as tools
plt.rcParams['font.sans-serif'] = ['SimHei']  # 步骤一（替换sans-serif字体）
plt.rcParams['axes.unicode_minus'] = False   # 步骤二（解决坐标轴负数的负号显示问题）

Populating the interactive namespace from numpy and matplotlib


# 数据获取

In [2]:
root_path = r"E:\data\01_antenna_self_optimize\new_method"
# 连接数据库
conn = psycopg2.connect(database='feeder', user='feeder_dev',
                        password='feeder_dev', host='192.168.101.66',
                        port='5432')

# 读取工参数据
#query = "SELECT * FROM test_mdt WHERE longitude=119.30127 AND latitude = 26.1057"
#mdt = pd.read_sql_query(query, conn)
#mdt = mdt.sort_values(['s_ci'])
query1 = "SELECT ci, longitude, latitude FROM sm_cl_location"
df_cell_info = pd.read_sql_query(query1, conn)

# 读取划定区域数据
# 选取划定区域 region_1 经度最小值 最大值 维度最小值 最大值
region_1 = [119.30127, 119.33075, 26.1057, 26.13431]
fold_name = '-'.join(list(map(str, region_1)))
project_path = os.path.join(root_path, fold_name)

if not os.path.exists(project_path):
    os.makedirs(project_path)
    
if os.path.exists(os.path.join(project_path, "df_selected_area.csv")):
    print("已经存在，直接读入")
    df_selected_area = pd.read_csv(os.path.join(project_path, "df_selected_area.csv"))
else:
    #query = "SELECT s_ci, longitude, latitude, s_rsrp FROM test_mdt WHERE\
    #        (longitude BETWEEN {} AND {}) AND (latitude BETWEEN {} AND {})\
    #            ".format(region_1[0], region_1[1], region_1[2], region_1[3])

    #df_selected_area = pd.read_sql_query(query, conn)
    #df_selected_area.to_csv(r"E:\01_work_documents\2018-08\df_selected_area.csv")
    df_selected_area = pd.read_csv(r"E:\01_work_documents\2018-08\df_selected_area.csv",
                                   index_col=0)

    # 仅保留小区的用户点位置种类大于20的小区 避免小区的用户点都集中在某一点的情况
    a = df_selected_area.groupby(['s_ci', 'longitude', 'latitude'], as_index=False).size().groupby(['s_ci']).size()
    b = a[a>5].index
    df_selected_area = df_selected_area[df_selected_area['s_ci'].isin(b)]
    # 仅保留用户数大于500的小区
    df_selected_area.groupby(['s_ci']).size().sort_values()
    a = df_selected_area.groupby(['s_ci']).size()
    b = a[a > 500].index
    df_selected_area = df_selected_area[df_selected_area['s_ci'].isin(b)]

    # 栅格化评估区域
    N = 10  # 经度
    M = 10  # 纬度
    lon_bins = np.arange(region_1[0], region_1[1] +
                         0.00002, (region_1[1] +
                         0.00001 - region_1[0]) / N)

    lat_bins = np.arange(region_1[2], region_1[3] + 0.00002, (region_1[3] +
                                                              0.00001 - region_1[2]) / M)

    df_selected_area['grid_num'] = np.nan

    grid_num = 1

    for i in range(0, len(lon_bins) - 1):
        for j in range(0, len(lat_bins) - 1):
            conds1 = (df_selected_area.longitude >= lon_bins[i]) & (df_selected_area.longitude
                                                                < lon_bins[i + 1])
            conds2 = (df_selected_area.latitude >= lat_bins[j]) & (df_selected_area.latitude
                                                               < lat_bins[j + 1])
            df_selected_area.loc[(conds1 & conds2), 'grid_num'] = grid_num
            grid_num += 1
    df_selected_area.to_csv(os.path.join(project_path, "df_selected_area.csv"), index=0)

已经存在，直接读入


In [4]:
num_liantiao_limit = 1  # 每个问题小区选取的联调小区个数上限
scope_liantiao = 0.5  # 问题小区扩充多少km为联调小区
tv_weak_cell = 0.005  # 确定弱覆盖小区对应的比率
scope_affected = 0.5  # 联调小区扩充多少km为受影响区域
n_neighbors = 10  # knn回归确定的邻居个数

# 确定待调小区

In [5]:
lst_cells_to_tune = []  # 待调小区 mdt格式

# 确定问题小区
df_selected_area['weak_flag'] =  df_selected_area['s_rsrp'].apply(lambda x: 1 if x < -120 else 0)
a1 = df_selected_area.groupby(['s_ci']).weak_flag.mean()
a2 = a1[a1 > tv_weak_cell]
lst_weak_cell_mdt = a2.index.tolist()
lst_weak_cell_sm = [tools.mdt2smcl(i) for i in lst_weak_cell_mdt]
lst_cells_to_tune.extend(lst_weak_cell_mdt)  # 将问题小区添加到待调小区列表中

# 根据问题小区确定联调小区 
lst_liantiao_mdt = []
for i in range(len(lst_weak_cell_mdt)):
    df_weak_cell = df_selected_area[df_selected_area['s_ci'] == lst_weak_cell_mdt[0]]
    lat_db = df_weak_cell['latitude'].min() - tools.km2lat(km=scope_liantiao)
    lat_ub = df_weak_cell['latitude'].max() + tools.km2lat(km=scope_liantiao)
    lon_lb = df_weak_cell['longitude'].min() - tools.km2lon(lat=(lat_db + lat_ub) / 2, km=scope_liantiao)
    lon_rb = df_weak_cell['longitude'].max() + tools.km2lon(lat=(lat_db + lat_ub) / 2, km=scope_liantiao)
    cond1 = df_selected_area['longitude'].between(lon_lb, lon_rb)
    cond2 = df_selected_area['latitude'].between(lat_db, lat_ub)
    df_liantiao_area = df_selected_area[cond1 & cond2]
    # 仅选取用户数排名后五的小区
    a1 = df_liantiao_area['s_ci'].value_counts() / len(df_liantiao_area)
    a1 = a1.sort_values(ascending=True)
    a1 = list(set(a1.index.tolist()) - set(lst_weak_cell_mdt))
    if len(a1) > num_liantiao_limit:
        a1 = a1[:num_liantiao_limit]
    lst_liantiao_mdt.extend(a1)

lst_cells_to_tune.extend(lst_liantiao_mdt)  # 将问题小区添加到待调小区列表中
lst_cells_to_tune = list(set(lst_cells_to_tune))

# 根据待调小区确定获取受影响区域
df_cells_to_tune = df_selected_area[df_selected_area['s_ci'].isin(lst_cells_to_tune)]
lat_db = df_cells_to_tune['latitude'].min() - tools.km2lat(km=scope_affected)
lat_ub = df_cells_to_tune['latitude'].max() + tools.km2lat(km=scope_affected)
lon_lb = df_cells_to_tune['longitude'].min() - tools.km2lon(lat=(lat_db + lat_ub) / 2, km=scope_affected)
lon_rb = df_cells_to_tune['longitude'].max() + tools.km2lon(lat=(lat_db + lat_ub) / 2, km=scope_affected)
cond1 = df_selected_area['longitude'].between(lon_lb, lon_rb)
cond2 = df_selected_area['latitude'].between(lat_db, lat_ub)
df_affected_area = df_selected_area[cond1 & cond2]

print("问题小区列表：", lst_weak_cell_mdt)
print("联调小区列表：", lst_cells_to_tune)

# 调整

In [21]:
lst_degrees = np.random.uniform(np.pi / 18, np.pi / 3, len(lst_cells_to_tune))  # 确定一个待调整度数
# copy划定区域数据 在副本上将不断进行删减 删除旋转后丧失的用户点 增加旋转后新增的用户点
df_selected_area_curr = df_selected_area.copy()
del df_selected_area_curr['weak_flag']
df_selected_area_curr['is_est'] = 0

In [54]:
lst_degrees

array([0.26813571, 0.71884423, 0.83868407, 0.37884337, 0.18850056,
       0.48079477, 0.37439089])

In [57]:
def fitness_func(*X):
    df_selected_area_tuned = tools.estimate_rsrp_on_given_degrees(df_selected_area, df_selected_area_curr, df_affected_area, df_cell_info, lst_cells_to_tune, X, project_path)

    # 计算适应度函数值
    df_selected_area_tuned['weak_flag'] =  df_selected_area_tuned['s_rsrp'].apply(lambda x: 1 if x < -120 else 0)
    c1 = df_selected_area_tuned.groupby(['grid_num']).weak_flag.mean()  # 每个小区的弱覆盖率
    c2 = len(c1[c1 > 0.0005])  # 弱覆盖小区个数
    fv = - (c2 + c1.mean())
    return fv

In [56]:
from algolib.optimization import Genetic_algorithm
bounds = [[- np.pi / 3, np.pi / 3]] * len(lst_cells_to_tune)
ga = Genetic_algorithm(bounds, fitness_func)

In [11]:
# 确定评估区域中各栅格的弱覆盖率 设每个栅格弱覆盖用户占比达到0.005视为 弱覆盖栅格
b1 = df_selected_area.groupby(['grid_num']).weak_flag.mean()
b2 = len(b1[b1 > 0.0005])

In [47]:
df_selected_area_tuned = tools.estimate_rsrp_on_given_degrees(df_selected_area, df_selected_area_curr, df_affected_area, df_cell_info, lst_cells_to_tune, lst_degrees, project_path)

0
1
2
3
4
5
6
7
8
9
10


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [ ]:
# 计算适应度函数值
# 确定评估区域中各栅格的弱覆盖率 设每个栅格弱覆盖用户占比达到0.005视为 弱覆盖栅格
b1 = df_selected_area.groupby(['grid_num']).weak_flag.mean()
b2 = len(b1[b1 > 0.0005])

df_selected_area_tuned['weak_flag'] =  df_selected_area_tuned['s_rsrp'].apply(lambda x: 1 if x < -120 else 0)
c1 = df_selected_area_tuned.groupby(['grid_num']).weak_flag.mean()
c2 = len(c1[c1 > 0.0005])
# df_final['weak_flag_tuned'] = df_final['s_rsrp_es'].apply(lambda x: 1 if x <
#                                                          -120 else 0)

# c1 = df_final.groupby(['grid_num']).weak_flag_tuned.mean()
# #    c2 = - c1.sum()  # c1.sum()越小越好 则 -c1.sum()越大越好
# c2 = len(c1[c1 > 0.005])
# print("调整前弱覆盖栅格个数：{} \n \b调整后弱覆盖栅格个数：{}".format(b2, c2))

In [60]:
optimalSolution, optimalValue = ga.fit(20)

当前迭代次数 0
当前迭代下的适应度函数值为:  [[8.51799018e-06]
 [4.68029311e-07]
 [7.06628226e-06]
 [8.51799018e-06]
 [7.02170272e-06]
 [8.42169410e-06]
 [1.69493366e-06]
 [3.04324193e-06]
 [7.06628226e-06]
 [8.56256972e-06]]
当前迭代次数 1
当前迭代下的适应度函数值为:  [[6.81947412e-06]
 [1.53633576e-06]
 [7.01901061e-06]
 [7.12995209e-06]
 [8.91044478e-06]
 [7.01939112e-06]
 [7.12995209e-06]
 [6.81958401e-06]
 [8.91044478e-06]
 [7.12995209e-06]]
当前迭代次数 2
当前迭代下的适应度函数值为:  [[6.75274478e-06]
 [7.04124244e-06]
 [7.03411081e-06]
 [6.78645754e-06]
 [7.03980463e-06]
 [6.95656867e-06]
 [5.32464241e-06]
 [7.04124244e-06]
 [6.95557861e-06]
 [6.74719420e-06]]
当前迭代次数 3
当前迭代下的适应度函数值为:  [[8.35281990e-06]
 [7.02061050e-06]
 [1.66075313e-06]
 [6.96477922e-06]
 [8.31832621e-06]
 [1.66075313e-06]
 [6.91214425e-06]
 [6.96477922e-06]
 [7.85316968e-06]
 [7.85316968e-06]]
当前迭代次数 4
当前迭代下的适应度函数值为:  [[9.70112881e-07]
 [7.13346070e-06]
 [6.94597824e-06]
 [6.94597824e-06]
 [1.63405048e-06]
 [6.94597824e-06]
 [7.12036477e-06]
 [1.88453493e-07]
 [6.773

In [61]:
optimalSolution

array([-0.76697329, -1.00552186, -1.00552186, -1.00552186, -1.00552186,
       -1.00552186, -1.00552186, -1.00552186])